1) 티커 가져오기 - done
2) Initialize DB
    - Last 30 minute prices만 필요
3) Update
    - 시간마다 ticker가져와서 업데이트?
4) Calculate
5) If triggered, check deposit & withdrawal availability

일정부분은 업비트에 두기.

In [59]:
import pandas as pd
import time
import asyncio, traceback

from Data.UpbitDownloader import UpbitDownloader 
from Data.BinanceDownloader import BinanceDownloader
import creds

class Arbitrage_Bot ():

    def __init__ (self):

        self.upbit = UpbitDownloader(rotate_ip=True)
        self.binance = BinanceDownloader()
        self.lookback = 6
        self.thres_prem = 0.03
        self.fx = 1350
        self.thres_vol = 300 # unit = $k/5min
        self.base_tickers = ['BTC', 'SOL'] # USDT to be manually added. Leave it blank if you only want to see USDT

        # timeframe of 5 minutes is hardcoded in.


    async def initialize(self):
        
        # Set tickers to track
        await self.update_tickers()

        # Set initial upbit ohlcv
        self.upbit_quotes = await self.upbit.get_multi_ohlcv(self.tickers, minutes = 5, num_bars = self.lookback, price_type ='Open', rotate_ip = True)
        self.upbit_bases = await self.upbit.get_multi_ohlcv(self.base_tickers + ['USDT'], minutes = 5, num_bars = self.lookback, price_type ='Open', rotate_ip = True)
        
        self.bin_quotes = await self.binance.get_multi_ohlcv(self.tickers, 'spot', '5m', self.lookback, price_type ='Open' )
        self.bin_bases = await self.binance.get_multi_ohlcv(self.base_tickers, 'spot', '5m', self.lookback, price_type ='Open' )
        self.bin_bases['USDT'] = 1 # Manually adding USDT

        if not self.upbit_quotes.index.equals(self.bin_quotes.index) or not self.bin_quotes.index.equals(self.upbit_bases.index):
            raise ValueError("arb: index unmatched")
        upbit_quote_current  = await self.upbit.get_current_prices(self.tickers)
        self.avg_vol = upbit_quote_current['acc_trade_price_24h']/self.fx/1e3 
        
        # Calc Premium
        await self.calc_premium()
        print(self.avg_premium.sort_values().iloc[:6], "\n", self.avg_premium.sort_values().iloc[-6:])
        
        # Set referesh times
        current_time = pd.to_datetime(time.time(), unit='s', utc=True)
        self.next_time_tickers = current_time.normalize() + pd.Timedelta(hours = current_time.hour + 1)
        self.next_time_prices = self.upbit_quotes.index[-1] + pd.Timedelta(minutes = 5)

        print("initialized successfully.")
        
    async def calc_premium (self):
        
        premiums = []
        quote_ratio = (self.upbit_quotes / self.bin_quotes)
        base_ratio = (self.upbit_bases / self.bin_bases)
        for base in base_ratio:
            premium = quote_ratio.div(base_ratio.loc[:,base], axis = 0) - 1
            premium.columns = [col + f"-{base}" for col in premium.columns]
            premiums.append(premium)
        self.premium = pd.concat(premiums, axis=1)
        self.avg_premium = self.premium.mean(axis=0)


    async def update_tickers (self):
        
        # Spot
        up_spot = await self.upbit.get_spot_tickers()
        bn_spot, bn_futures = await self.binance.get_tickers_with_both_spotNfutures()
        tickers = [ticker for ticker in up_spot if ticker in bn_spot]
        self.tickers = tickers
        
        # Binance Futures
        bin_fut_dict = dict(zip(bn_spot, bn_futures))
        self.tickers_bn_fut = [bin_fut_dict[ticker] for ticker in self.tickers]


    async def update_data (self):

        current_time = pd.to_datetime(time.time(), unit='s', utc=True)

        ## Upbit Needs Extra Code because their get_multi_ohlcv is too slow, so need to do it manually with get_current_prices        
        if current_time >= self.next_time_prices:

            # Upbit Quote
            upbit_quote_current  = await self.upbit.get_current_prices(self.tickers) # Adding in USDT Separately
            for ticker in upbit_quote_current.index:
                self.upbit_quotes.at[self.next_time_prices, ticker] = upbit_quote_current.loc[ticker, 'trade_price']
                self.upbit_quotes = self.upbit_quotes.iloc[-self.lookback:]
            self.avg_vol = upbit_quote_current['acc_trade_price_24h']/self.fx/1e3 


            # Upbit Base
            upbit_base_current  = await self.upbit.get_current_prices(self.base_tickers + ['USDT']) # Adding in USDT Separately
            for ticker in upbit_base_current.index:
                self.upbit_bases.at[self.next_time_prices, ticker] = upbit_base_current.loc[ticker, 'trade_price']
                self.upbit_bases = self.upbit_bases.iloc[-self.lookback:]                


            # Binance Quote & Spot
            self.bin_quotes = await self.binance.get_multi_ohlcv(self.tickers, 'spot', '5m', self.lookback, price_type ='Open' )
            self.bin_bases = await self.binance.get_multi_ohlcv(self.base_tickers, 'spot', '5m', self.lookback, price_type ='Open' )
            self.bin_bases['USDT'] = 1
            
            # self.df_bin_fut = await self.binance.get_multi_ohlcv(self.tickers_bn_fut, 'futures', '5m', self.lookback, price_type ='Open')
            
            # Error check
            if not self.upbit_quotes.index.equals(self.upbit_bases.index) or not self.upbit_bases.index.equals(self.bin_quotes.index) or not self.bin_quotes.index.equals(self.bin_bases.index):
                print(self.upbit_quotes.index)
                print(self.upbit_bases.index)
                print(self.bin_quotes.index)
                print(self.bin_bases.index)
                raise ValueError("Index mismatch with data update")
            
            # Calculate Premium
            await self.calc_premium()
                                    

    async def check_signals (self):

        avg_vol_expanded = pd.concat([self.avg_vol]*(1+len(self.base_tickers)),axis=1) # 1 manually added for USDT

        price_condition = (abs(self.avg_premium) > self.thres_prem).values
        volume_condition = (avg_vol_expanded > self.thres_vol).values
        
        avg_prem_filtered = self.avg_premium[price_condition & volume_condition]
        last_prem_filtered = self.premium.iloc[-1,:][price_condition & volume_condition]
        vol_filtered = avg_vol_expanded[price_condition & volume_condition]
        
        if len(avg_prem_filtered) > 1:
            msg = f'''
                    avg: {str(round(avg_prem_filtered,3))}
                    vol: {str(round(vol_filtered,3))}
                    last: {str(round(last_prem_filtered,0))}
                    '''
            creds.send_pushover(msg, jihoon=False)


        ''' 
        Below is for test. To be deleted
        '''

        price_condition = (abs(self.avg_premium) > self.thres_prem/3).values
        volume_condition = (avg_vol_expanded > self.thres_vol/2).values
        
        avg_prem_filtered = self.avg_premium[price_condition & volume_condition]
        last_prem_filtered = self.premium.iloc[-1,:][price_condition & volume_condition]
        vol_filtered = avg_vol_expanded[price_condition & volume_condition]
        
        if len(avg_prem_filtered) > 1:
            msg = f'''
                    avg: {str(round(avg_prem_filtered,3))}
                    vol: {str(round(vol_filtered,3))}
                    last: {str(round(last_prem_filtered,0))}
                    '''
            await creds.send_telegram(msg)
 

    async def main(self):

        await self.initialize()
        time_margin = pd.Timedelta(seconds=1)

        while True:
            current_time = pd.to_datetime(time.time(), unit='s', utc=True)
            if current_time > self.next_time_prices + time_margin:    
                await self.update_data()
                await self.check_signals()
                self.next_time_prices += pd.Timedelta(minutes = 5)

            current_time = pd.to_datetime(time.time(), unit='s', utc=True)
            if current_time > self.next_time_tickers + time_margin:
                await self.update_tickers()
                self.next_time_tickers += pd.Timedelta(minutes = 30)

            await asyncio.sleep(2)

    async def close(self):
        await self.upbit.ccxt.close()
        await self.binance.ccxt.close()
        print("closed successfully")


# if __name__ == '__main__':
#     try:
#         arb = Arbitrage_Bot()
#         asyncio.run(arb.main())
#     except Exception as e:
#         error_message = f"{e}\n\nTraceback:\n{traceback.format_exc()}"
#         print(error_message)
#         asyncio.run(creds.send_telegram(str(e)))
#     except KeyboardInterrupt:
#         print("keyboard interruption")
#     finally:
#         asyncio.run(arb.close())




In [60]:
await bot.close()

closed successfully


In [61]:
bot = Arbitrage_Bot()

Starting API gateway in 1 regions.
Using 1 endpoints with name 'https://api.upbit.com - IP Rotate API' (0 new).


In [62]:
await bot.initialize()

LSK-USDT      -0.004626
LSK-BTC       -0.004612
LSK-SOL       -0.003808
PENDLE-USDT   -0.001964
LOOM-USDT     -0.001963
PENDLE-BTC    -0.001951
dtype: float64 
 ONT-SOL     0.001826
ANKR-SOL    0.001855
IMX-SOL     0.002057
CELO-SOL    0.002115
GRT-SOL     0.002127
BLUR-SOL    0.002283
dtype: float64
initialized successfully.


In [68]:
bot.avg_premium

BTC-BTC        0.000000
ETH-BTC        0.000033
NEO-BTC       -0.000166
MTL-BTC       -0.000190
XRP-BTC       -0.000503
                 ...   
STG-USDT      -0.000365
ZRO-USDT      -0.001335
JUP-USDT      -0.000863
ENS-USDT      -0.001042
PENDLE-USDT   -0.001964
Length: 243, dtype: float64

In [82]:
await bot.update_data()

In [72]:
bot.avg_premium

BTC-BTC        0.000000
ETH-BTC        0.000069
NEO-BTC       -0.000487
MTL-BTC       -0.000372
XRP-BTC       -0.000201
                 ...   
STG-USDT      -0.000485
ZRO-USDT      -0.001530
JUP-USDT      -0.000763
ENS-USDT      -0.000778
PENDLE-USDT   -0.001847
Length: 243, dtype: float64

In [83]:
bot.avg_premium

BTC-BTC        0.000000
ETH-BTC       -0.000037
NEO-BTC       -0.000365
MTL-BTC       -0.000824
XRP-BTC       -0.000448
                 ...   
STG-USDT      -0.000446
ZRO-USDT      -0.001241
JUP-USDT      -0.001293
ENS-USDT      -0.000829
PENDLE-USDT   -0.002100
Length: 243, dtype: float64

In [84]:
bot.avg_premium.sort_values(ascending=False)

BLUR-SOL       0.002064
GRT-SOL        0.002010
EGLD-SOL       0.001975
ANKR-SOL       0.001958
CELO-SOL       0.001795
                 ...   
PENDLE-BTC    -0.002031
PENDLE-USDT   -0.002100
LSK-SOL       -0.003958
LSK-BTC       -0.004732
LSK-USDT      -0.004801
Length: 243, dtype: float64

In [16]:
bot.premium

,BTC,ETH,NEO,MTL,XRP,ETC,QTUM,LSK,STEEM,XLM,...,MINA,ASTR,ID,PYTH,AUCTION,STG,ZRO,JUP,ENS,PENDLE
Time,,,,,,,,,,,,,,,,,,,,,
2024-08-08 02:20:00+00:00,-0.000959,-0.000989,-0.000563,-0.002693,-0.000560,-0.000570,-0.000209,-0.005199,-0.000821,-0.000727,...,-0.000854,-0.001432,-0.001964,0.000562,-0.001094,-0.000761,-0.000665,0.000655,0.000625,-0.002951
2024-08-08 02:25:00+00:00,-0.000769,-0.000844,-0.001632,-0.000693,-0.000831,-0.000709,0.000178,-0.004846,-0.001582,-0.000408,...,-0.001464,-0.004635,-0.002889,0.001695,-0.003784,0.000590,-0.000143,-0.000866,-0.000968,-0.002282
2024-08-08 02:30:00+00:00,-0.001551,-0.001283,-0.003157,-0.000301,-0.001816,-0.001395,-0.002138,-0.006201,-0.004725,-0.001787,...,-0.003674,-0.001552,-0.001748,-0.000575,-0.002979,-0.001492,-0.002864,-0.000638,-0.001066,-0.003044
2024-08-08 02:35:00+00:00,-0.001277,-0.002072,0.000471,0.001526,-0.002013,-0.001194,0.000332,-0.005657,-0.001481,-0.000509,...,-0.000956,-0.001721,0.000893,-0.001691,-0.001611,-0.000100,-0.000900,-0.002042,-0.001531,-0.002194
2024-08-08 02:40:00+00:00,-0.000605,-0.000355,-0.002062,-0.000336,-0.001698,-0.000139,-0.000453,-0.007012,-0.004768,-0.001758,...,NaN,-0.001703,-0.001159,0.000370,-0.000941,-0.000885,-0.001204,0.000075,-0.000122,-0.002604
2024-08-08 02:45:00+00:00,-0.000888,-0.001325,-0.003587,-0.002704,-0.000474,-0.001293,0.000486,-0.002034,-0.002234,0.000431,...,-0.002494,-0.000541,-0.002120,-0.000694,-0.002675,-0.001040,-0.000822,-0.001699,-0.002664,-0.003880


In [95]:
price_condition = (abs(bot.avg_premium) > 0.002).values
volume_condition = (bot.upbit_quote_current['acc_trade_price_24h']/bot.fx/1e3 > bot.thres_vol).values
filtered = bot.avg_premium[price_condition & volume_condition]

In [96]:
filtered

ETC      0.003326
LSK     -0.002094
EOS      0.002195
KNC      0.002114
THETA    0.002088
1INCH    0.002367
dtype: float64